# Mission One, Part l (Visualization)

In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [2]:
# loading csv dataset as a DataFrame
df_original = pd.read_csv("210619monatszahlenjuni2021monatszahlen2106verkehrsunfaelle.csv") 

df = df_original[['MONATSZAHL', 'AUSPRAEGUNG', 'JAHR', 'MONAT', 'WERT']]

# visualizing categories and accident types
unique_categories = df['MONATSZAHL'].unique()
unique_accident_types = df['AUSPRAEGUNG'].unique()
fig = go.Figure(data=[go.Table(header=dict(values=['Category', 'Accident Type']),
                               cells=dict(values=[unique_categories, unique_accident_types]))]
                )
fig.show()

In [3]:
# creating list of years excluding 2021
list_of_years = df['JAHR'].unique()[1:]

### Visualization of accidents due to "Alkoholunfälle"

In [4]:
# in this code cell, we query rows based on Alkoholunfälle category for each accident type
alko_total = df.loc[(df['MONATSZAHL'] == 'Alkoholunfälle') & (df['AUSPRAEGUNG'] == 'insgesamt') & (df['MONAT'] == 'Summe')]
alko_verl = df.loc[(df['MONATSZAHL'] == 'Alkoholunfälle') & (df['AUSPRAEGUNG'] == 'Verletzte und Getötete') & (df['MONAT'] == 'Summe')]

# calculating unspecified type of accidents by subtracing from total value(named others) 
others = alko_total['WERT'] - alko_verl['WERT'].values

fig = go.Figure(data=[
                      go.Bar(name='Others', x=list_of_years, y=others),
                      go.Bar(name='Verletzte und Getötete', x=list_of_years, y=alko_verl['WERT'])
                      ])

fig.update_layout(barmode='stack',
                  title_text='Number of Accidents due to "Alkoholunfälle" per Accident Type 2000-2020')      
fig.show()


### Visualization of accidents due to "Verkehrsunfälle"


In [5]:
# in this code cell, we query rows based on Verkehrsunfälle category for each accident type
verk_total = df.loc[(df['MONATSZAHL'] == 'Verkehrsunfälle') & (df['AUSPRAEGUNG'] == 'insgesamt') & (df['MONAT'] == 'Summe')]
verk_verl = df.loc[(df['MONATSZAHL'] == 'Verkehrsunfälle') & (df['AUSPRAEGUNG'] == 'Verletzte und Getötete') & (df['MONAT'] == 'Summe')]
verk_mit = df.loc[(df['MONATSZAHL'] == 'Verkehrsunfälle') & (df['AUSPRAEGUNG'] == 'mit Personenschäden') & (df['MONAT'] == 'Summe')]

# calculating unspecified type of accidents by subtracing from total value(named others) 
others = verk_total['WERT'] - (verk_verl['WERT'] + verk_mit['WERT'].values).values

fig = go.Figure(data=[
                      go.Bar(name='Others', x=list_of_years, y=others),
                      go.Bar(name='Verletzte und Getötete', x=list_of_years, y=verk_verl['WERT']),
                      go.Bar(name='mit Personenschäden', x=list_of_years, y=verk_mit['WERT'])
                      ])
fig.update_layout(barmode='stack',
                  title_text='Number of Accidents due to "Verkehrsunfälle" per Accident Type 2000-2020')
fig.show()